In [5]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [100]:
import pinout
from lsm6dso import LSM6DSO
from machine import I2C, Pin, Timer
import time
from common import task

In [102]:
i2c = I2C(0, scl=Pin(pinout.scl), sda=Pin(pinout.sda), freq=1000000)
mems = LSM6DSO(i2c)
mems.scale_a('16g')

acc_filename = '/user/acc.csv'


def build_cb(buf):
    def measure_store(tim):
        t = time.ticks_us()
        val = mems.get_a()
    
        buf.append({
            'dt': t,
            'acc': tuple(val)
        })
    return measure_store
        
@task('1: Acc collecting')
def task_1_acc_collecting(duration, freq):
    with open(acc_filename, 'w') as f:
        f.write("#dt, x, y, z\n")
    
        buf = list()
    
        tim = Timer(0)
        tim.init(period=int(1000/freq), mode=Timer.PERIODIC, callback=build_cb(buf))
    
        time.sleep(duration)
        tim.deinit()
    
        for m in buf:
            f.write(f"{m['dt']}, {m['acc'][0]}, {m['acc'][1]}, {m['acc'][2]}\n")
    
    print(f'Data written to {acc_filename}')
    
    
task_1_acc_collecting(30, 30)

MemoryError: memory allocation failed, allocating 512 bytes
